<a href="https://colab.research.google.com/github/linus3003/PropVisionDashboard/blob/master/PropML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q xlrd
!git clone https://github.com/linus3003/AI-talents.git
!ls AI-talents
%cd AI-talents

Cloning into 'AI-talents'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (81/81), done.
alldata0301.csv     alldata_final.csv	  linksandtext.csv	 links.csv
alldata.csv	    date_translation.csv  linksandtextfinal.csv  links_new.csv
alldata_final2.csv  linksandtext_2.csv	  linksandtext_new.csv	 README.md
/content/AI-talents


In [ ]:
import pandas as pd
import regex as re
import numpy as np
pd.set_option('display.max_colwidth', None)

In [ ]:
import os
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import set_config


In [ ]:
!pip install pgeocode
import pgeocode

In [ ]:
df = pd.read_csv('alldata.csv', index_col = 0)
df.reset_index(inplace=True,drop=True)

In [ ]:
df[:10]

In [ ]:
data = df.drop([cleantext','point','allprice','allrooms','allsqft','latitude','longitude','address_sc','address_clean','unit_id'], axis=1)

In [ ]:
data.columns.to_list()

['index',
 'url',
 'dev_status',
 'is_unit',
 'location',
 'price',
 'sqft',
 'rooms',
 'wohntyp',
 'price-pred',
 'region',
 'is_lk',
 'diff_from_prediction',
 'schools5km']

In [ ]:
#region = re.compile("[\,] ([\S]+)[\,] [\d]+[\,] Deutschland")
plz = re.compile("[\,] [\S]+[\,] ([\d]+)[\,] Deutschland")

In [ ]:
#data['region']= [region.search(str(x)).group(1) if city.search(str(x)) is not None else np.NaN
#               for x in data['location']]

In [ ]:
data['plz']= [plz.search(str(x)).group(1) if plz.search(str(x)) is not None else np.NaN
               for x in data['location']]

In [ ]:
#get community name from zip-code from pgeocode/nominatim api
data['city']= [nomi.query_postal_code(plz).community_name if pd.isna(plz)==False else "" for plz in data['plz']] 

In [ ]:
data['city'].unique()

In [ ]:
landkreis = re.compile("Landkreis")

In [ ]:
data['is_lk'] = [True if landkreis.search(str(ct)) is not None else False for ct in data['city']]

In [ ]:
data.drop(['location'],axis=1,inplace=True)

In [ ]:
citycount = data.groupby('city')['url'].nunique()

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
#convert city to other if it occurs less than 10 times
data=data.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'other') if x.name=='city' else x)

In [ ]:
citycount

city
                                      534
Augsburg                               32
Bayreuth                               16
Berchtesgadener Land                    5
Berlin, Stadt                         695
Böblingen                              34
Düren                                   6
Düsseldorf, Stadt                      35
Erding                                 21
Essen, Stadt                           47
Esslingen                              12
Frankfurt am Main, Stadt              100
Fürstenfeldbruck                       27
Fürth                                   8
Groß-Gerau                             17
Göppingen                               9
Hamburg, Freie und Hansestadt         147
Ingolstadt                             23
Karlsruhe                              10
Kaufbeuren                              7
Kreis Ostholstein                      11
Kreis Pinneberg                        18
Kreis Segeberg                         12
Kreis Stormarn               

In [ ]:
data.columns

Index(['index', 'url', 'dev_status', 'is_unit', 'location', 'price', 'sqft',
       'rooms', 'wohntyp', 'price-pred', 'region', 'is_lk',
       'diff_from_prediction', 'schools5km'],
      dtype='object')

In [ ]:
data.wohntyp.unique()

array(['Eigentumswohnung', 'Doppelhaushalfte', 'Apartment',
       'Etagenwohnung', 'Dachgeschosswohnung', 'Erdgeschosswohnung',
       'Reihenhaus', nan, 'Penthouse', 'Maisonettewohnung',
       'Terrassenwohnung', 'Einfamilienhaus', 'Souterrainwohnung',
       'Reihenendhaus', 'Loft', 'Reihenmittelhaus', 'Galeriewohnung',
       'Reiheneckhaus', 'Bungalow', 'Stadthaus', 'Townhouses'],
      dtype=object)

In [ ]:
data = data.drop(['price-pred','index','diff_from_prediction','location','url', 'is_unit'], axis=1)

In [ ]:
X=data.dropna(subset=['price'])

In [ ]:
Y= X.pop('price')

In [ ]:
num_feats = X.select_dtypes(include=['float64', 'int64']).columns
cat_feats = X.select_dtypes('object').columns
bin_feats = ['schools5km']


In [ ]:
cat_feats

Index(['dev_status', 'wohntyp', 'region'], dtype='object')

In [ ]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.1, random_state =42)

In [ ]:
X_train.dtypes

dev_status     object
sqft          float64
rooms         float64
wohntyp        object
region         object
is_lk            bool
schools5km    float64
dtype: object

In [ ]:
params = {
          'xgb__learning_rate': [0.05, 0.1, 0.15, 0.2, 0.3], 
          'xgb__max_depth': [4, 5, 6,],
          'xgb__min_child_weight': [1, 5, 10],
          'xgb__gamma': [0.5, 1, 1.5, 2, 5],
          'xgb__colsample_bytree': [0.4,0.6, 0.8, 1.0],
          'xgb__subsample': [0.6, 0.8, 1.0],
          'xgb__n_estimators' : [400]
          }

In [ ]:
numeric_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='median')),
      ('scaler', StandardScaler())])

cat_bin = Pipeline(steps= [
                           
      ('imputer', SimpleImputer(strategy='median'))          ,              
      ('binning', KBinsDiscretizer(n_bins = 5, strategy='quantile'))                      

])


categorical_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='constant')),
      ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
      transformers=[
          ('num', numeric_transformer, num_feats),
          ('bin', cat_bin, bin_feats),
          ('cat', categorical_transformer, cat_feats)])

fullpipe = Pipeline([
      ('prep', preprocessor),
      ('xgb', xgb.XGBRegressor(objective='reg:squarederror'))
])

In [ ]:
#Random search Crossvalidation
search = RandomizedSearchCV(n_iter=400, estimator = fullpipe, param_distributions=params,cv=10, return_train_score=True )
search.fit(X_train,y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('prep',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('num',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                                            

In [ ]:
print(search.best_estimator_)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                     

In [ ]:
print(search.best_score_)

0.7257239789888235


In [ ]:
best_est = search.best_estimator_

best_est.fit(X_train, y_train)

# predict
train_preds = best_est.predict(X_train)
test_preds = best_est.predict(X_test)

In [ ]:
#save model
from joblib import dump, load
dump(best_est, 'model.joblib',) 

['model.joblib']

In [ ]:
from google.colab import files
files.download('model.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tempfile import mkdtemp
savedir = mkdtemp()
import os
filename = os.path.join(savedir, 'test.joblib')

In [ ]:
train_mae = mean_absolute_error(y_train,train_preds)
test_mae = mean_absolute_error(y_test,test_preds)
train_mse = mean_squared_error(y_train,train_preds)
test_mse = mean_squared_error(y_test,test_preds)

with 5kmschools binned



In [ ]:
print("train_mae: %f test_mae: %f" % (train_mae,test_mae))

train_mae: 74092.812718 test_mae: 117937.596821


without

In [ ]:
print("train_mae: %f test_mae: %f train mse %f test mse %f" % (train_mae,test_mae, train_mse, test_mse))

train_mae: 110081.172034 test_mae: 137530.861509 train mse 44041765957.161949 test mse 126616424990.404556


In [ ]:
len(data)

5206

In [ ]:
#retrain on full dataset
X = data
Y=X.pop('price')

In [ ]:
best_est.fit(X,Y)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                     

In [ ]:
predictions = best_est.predict(X)

In [ ]:
df['price-pred']=predictions

In [ ]:
df['region']=data['city']

In [ ]:
df['is_lk']=data['is_lk']

In [ ]:
df['diff_from_prediction']=(df['price']-df['price-pred'])/df['price']

In [ ]:
df.columns

Index(['url', 'text', 'cleantext', 'dev_status', 'address_sc', 'address_clean',
       'is_unit', 'location', 'point', 'latitude', 'longitude', 'allprice',
       'allrooms', 'allsqft', 'price', 'sqft', 'rooms', 'unit_id', 'wohntyp',
       'price-pred', 'region', 'is_lk'],
      dtype='object')

In [ ]:
df.to_csv('alldata.csv')

In [ ]:
wohntypen = df['wohntyp'].unique()

In [ ]:
wohntypen

array(['Eigentumswohnung', 'Doppelhaushalfte', 'Apartment',
       'Etagenwohnung', 'Dachgeschosswohnung', 'Erdgeschosswohnung',
       'Reihenhaus', nan, 'Penthouse', 'Maisonettewohnung',
       'Terrassenwohnung', 'Einfamilienhaus', 'Souterrainwohnung',
       'Reihenendhaus', 'Loft', 'Reihenmittelhaus', 'Galeriewohnung',
       'Reiheneckhaus', 'Bungalow', 'Stadthaus', 'Townhouses'],
      dtype=object)

In [ ]:
regions = df['region'].unique()

In [ ]:
regions

array(['Berlin, Stadt', 'Landkreis Oberhavel', 'Kreisfreie Stadt Potsdam',
       'other', 'Landkreis Heilbronn', '', 'Landkreis Potsdam-Mittelmark',
       'Kreisfreie Stadt Dresden', 'Landkreis Dahme-Spreewald',
       'Frankfurt am Main, Stadt', 'Berchtesgadener Land',
       'Kreisfreie Stadt München', 'Landkreis Rosenheim', 'Miesbach',
       'Landshut', 'Starnberg', 'Landkreis München',
       'Landkreis Wittenberg', 'Fürstenfeldbruck', 'Landkreis Ebersberg',
       'Landkreis Dachau', 'Kaufbeuren',
       'Landkreis Bad Tölz-Wolfratshausen', 'Erding',
       'Landkreis Landsberg am Lech', 'Wiesbaden, Landeshauptstadt',
       'Main-Kinzig-Kreis', 'Salzgitter, Stadt', 'Kreis Ostholstein',
       'Rheingau-Taunus-Kreis', 'Landkreis Mainz-Bingen',
       'Landkreis Aschaffenburg', 'Offenbach', 'Wetteraukreis',
       'Main-Taunus-Kreis', 'Groß-Gerau', 'Kreis Pinneberg',
       'Kreis Stormarn', 'Hamburg, Freie und Hansestadt',
       'Kreis Segeberg', 'Lübeck, Hansestadt', 'Mönchen

In [ ]:
#Create Dataset for Dash App
adf = pd.read_csv('alldata.csv', index_col=0)

In [ ]:
dashdata = adf.drop(['cleantext', 'address_sc','is_unit','point'], axis=1)

In [ ]:
dashdata['Name']= [url.replace("https://www.neubaukompass.de/neubau/","").replace("https://www.howoge.de/wohnungsbau/neubauprojekte/","").replace(".html","").replace("/"," ").replace("-"," ").split(" ") for url in dashdata.url]

In [ ]:
dashdata['Name']= [[t.capitalize() for t in text ] for text in dashdata['Name']]

In [ ]:
dashdata['Name']= [" ".join(text) for text in dashdata['Name']]

In [ ]:
dashdata['Price']= [ "€{:,.2f}".format(price) for price in dashdata['price']]

In [ ]:
dashdata['Predicted price']= [ "€{:,.2f}".format(price) for price in dashdata['price-pred']]

In [ ]:
dashdata['Living space'] = ["{:,.0f} m²".format(price) for price in dashdata['sqft']]

In [ ]:
dashdata['Status'] = dashdata['dev_status']

In [ ]:
dashdata['Number of rooms'] = dashdata['rooms']

In [ ]:
dashdata=dashdata.rename(columns={"wohntyp": "Wohntyp", "address_clean": "Address"})

In [ ]:
dashdata.to_csv('dashdata.csv')